<a href="https://colab.research.google.com/github/enigmatized/NoGambleNoFuture/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What I want to do is,
find correlation between volume and SD moves in price diff.
Also I want to start looking at 1min price diff and compare that on the 5min, 15min.
To find a pattern within that to trade on the 15.

Eventually I want to add in trading events.


1. See if the limit of low helps
2. See if the time frames on lower level help, not sure how that should be done exactly?
3. Also volume spikes, this is a weird one because there is an event spike that is not being considered here which I think is really important
4. Data

In [27]:
from collections import defaultdict
import requests
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time
from urllib.error import HTTPError
import statistics

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#
cur_working_directiory = os.getcwd()
# Change 
os.chdir("../../Data/")


CSVfiles = []
for file in os.listdir("./"):
  if file.endswith(".csv"):
    CSVfiles.append(file)
print(CSVfiles)


unixTimeConverter = lambda x : (datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S')
unixTimeConverterNoDateJustTime = lambda x: (datetime.fromtimestamp(x) - timedelta(hours=4)).time()


allDfs = []


for x in CSVfiles:

  dfTemp = pd.read_csv(x)
  dfTemp['estTime']  = dfTemp['time'].apply(unixTimeConverter)
  dfTemp['timeOnly'] = dfTemp['time'].apply(unixTimeConverterNoDateJustTime)

  dfTemp['candleHeight']  = dfTemp.apply( lambda row:    max(row['open'],  row['close']) - min(row['open'],  row['close']), axis = 1 )
  allDfs.append(dfTemp)

df = allDfs[0]
#This is for generating if profit
for i in range(1, 15):
  df['P/L' +str(i) +'Short_']  = ( df['close'] - df['close'].shift(-i) )
  df['P/L' +str(i) +'Long_']  =  (  df['close'].shift(-i) - df['close']     )
  df['P/L' +str(i) +'Short']  = ( df['close'] - df['close'].shift(i) )
  df['P/L' +str(i) +'Long']  =  (  df['close'].shift(i) - df['close']     )

df = df.dropna()


os.chdir(cur_working_directiory)

df.head()

['OANDA EURUSD, 5(2).csv']


,time,open,high,low,close,Volume,Volume MA,estTime,timeOnly,candleHeight,...,P/L12Short,P/L12Long,P/L13Short_,P/L13Long_,P/L13Short,P/L13Long,P/L14Short_,P/L14Long_,P/L14Short,P/L14Long
14,1681399800,1.10544,1.10549,1.10522,1.10528,290,545.80,2023-04-13 11:30:00,07:30:00,0.00016,...,-0.00052,0.00052,-0.00004,0.00004,-0.00008,0.00008,0.00021,-0.00021,0.00052,-0.00052
15,1681400100,1.10528,1.10590,1.10528,1.10572,343,523.55,2023-04-13 11:35:00,07:35:00,0.00044,...,0.00074,-0.00074,0.00065,-0.00065,-0.00008,0.00008,0.00066,-0.00066,0.00036,-0.00036
16,1681400400,1.10572,1.10594,1.10572,1.10580,350,503.60,2023-04-13 11:40:00,07:40:00,0.00008,...,0.00092,-0.00092,0.00074,-0.00074,0.00082,-0.00082,0.00042,-0.00042,0.00000,0.00000
17,1681400700,1.10580,1.10616,1.10574,1.10600,355,489.65,2023-04-13 11:45:00,07:45:00,0.00020,...,0.00126,-0.00126,0.00062,-0.00062,0.00112,-0.00112,0.00042,-0.00042,0.00102,-0.00102
18,1681401000,1.10600,1.10602,1.10520,1.10522,392,477.50,2023-04-13 11:50:00,07:50:00,0.00078,...,0.00053,-0.00053,-0.00036,0.00036,0.00048,-0.00048,-0.00040,0.00040,0.00034,-0.00034


In [37]:
d_counts_average_vol            = defaultdict(list)
d_counts_average_highLow_diff   = defaultdict(list)

df['closeOpenDelta']   = (abs( df['close'] - df['open'] )).copy()
df['CloseOpenDelta_']  = (df['close'] - df['open']).copy()

df['openLowDelta']   = (abs( df['open'] - df['low'] )).copy()
df['highLowDelta']  = (df['high'] - df['low']).copy()




for index, row in df.iterrows():
  #put volumes in d
  time_  = str(row['timeOnly'])
  volume = row['Volume']
  d_counts_average_vol[str(time_)].append(volume)


  #Collecting, using a dictionary of lists to collect the price deltas for each time
  price_delta = row['closeOpenDelta']
  d_counts_average_highLow_diff[str(time_)].append(price_delta)

df.head()
###############################################
##THIS use to be seperate cells, but now I just combined them
##
################################################
import statistics

std_values_with_time_as_keys = {}
mean_values_with_time_as_keys = {}

#for volume
std_values_with_time_as_keys_volume = {}
mean_values_with_time_as_keys_volume = {}

for k , v in d_counts_average_highLow_diff.items():

  # Calculate mean and standard deviation of the sample for price delta
  mean = statistics.mean(v)
  std_dev = statistics.stdev(v)
  std_values_with_time_as_keys[k] = std_dev
  mean_values_with_time_as_keys[k] = mean

  # # Calculate mean and standard deviation of the sample of volume
  # mean = statistics.mean(d_counts_average_vol[k])
  # std_dev = statistics.stdev(d_counts_average_vol[k])
  # std_values_with_time_as_keys_volume[k] = std_dev
  # mean_values_with_time_as_keys_volume[k] = mean

#####
#z-score manafacturing

df['z-score']= (df['close']).copy()
df['z-score-volume']= (df['close']).copy()


for index, row in df.iterrows():
  time_  = str(row['timeOnly'])
  mean_ = mean_values_with_time_as_keys[time_]
  std_ = std_values_with_time_as_keys[time_]
  #wow so baically shorting highs is a very profitbale strategy, who knew.
  #WHo can call the top type of thing.
  #But it does seem rather self, fullfilling, but this is very backwards data, you already know the answer.
  #Damn I feel stupid
  df.loc[index, 'z-score'] = (row['closeOpenDelta'] - mean)/ std_
  # row['z-score'] = (row['closeOpenDelta'] - mean)/ std_
  
  # mean_ = mean_values_with_time_as_keys_volume[time_]
  # std_ = std_values_with_time_as_keys_volume[time_]
  # df.loc[index, 'z-score-volume'] = (row['Volume'] - mean)/ std_

df.head(1)

/tmp/ipykernel_806325/4287134581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['closeOpenDelta']   = (abs( df['close'] - df['open'] )).copy()
/tmp/ipykernel_806325/4287134581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CloseOpenDelta_']  = (df['close'] - df['open']).copy()
/tmp/ipykernel_806325/4287134581.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,time,open,high,low,close,Volume,Volume MA,estTime,timeOnly,candleHeight,...,P/L14Short_,P/L14Long_,P/L14Short,P/L14Long,closeOpenDelta,CloseOpenDelta_,openLowDelta,highLowDelta,z-score,z-score-volume
14,1681399800,1.10544,1.10549,1.10522,1.10528,290,545.8,2023-04-13 11:30:00,07:30:00,0.00016,...,0.00021,-0.00021,0.00052,-0.00052,0.00016,-0.00016,0.00022,0.00027,-2.198284e+06,1.10528


In [35]:
df.head(1)
df['z-score'].describe()

count    1.675100e+04
mean    -3.717830e+06
std      1.878764e+06
min     -1.034130e+07
25%     -4.980262e+06
50%     -3.140228e+06
75%     -2.281627e+06
max     -5.039687e+05
Name: z-score, dtype: float64

In [38]:
""" 
This is a split test, curious to see if having a 
low openLowDelta, along with a 4+ z-score, will
increase a higher profitablity rate.
"""

totalProfShort = defaultdict(float)
totalProfLong  = defaultdict(float)
totalPro       = defaultdict(float)

#Note this is actually shorting, I am doing the opposite of the 
#Large directional move, hence mean reversion
totalProfLongList = defaultdict(list)
#Comparing this 
#to see if only buy when the open and low are close together
totalProfLongListWithLowOpenLowDelata = defaultdict(list)
totalProfLongWithLowOpenLowDelta      = defaultdict(float)
#
#to see if only buy when the open and low are FAR away
totalProfLongListWithHighOpenLowDelata = defaultdict(list)
totalProfLongWithHighOpenLowDelta      = defaultdict(float)
#
#volume filter
totalProfLongListWithHighVolume = defaultdict(list)
totalProfLongWithHighVolume      = defaultdict(float)
totalProfLongListWithLowVolume = defaultdict(list)
totalProfLongWithLowVolume      = defaultdict(float)



# allBigSDMoves
for index, row in df.iterrows():
  if row['z-score'] >= 3:
    if row['CloseOpenDelta_'] > 0:
      for i in range(1,15):
        lookup = 'P/L' + str(i) +'Short'
        totalProfLong[lookup] +=row[lookup]
        totalProfLongList[lookup].append(row[lookup])
        #print(row['CloseOpenDelta_'])
        if row['openLowDelta'] <= 0.00006:
          totalProfLongWithLowOpenLowDelta[lookup] +=row[lookup]
          totalProfLongListWithLowOpenLowDelata[lookup].append(row[lookup])
        if row['openLowDelta'] > 0.00006:
          totalProfLongWithHighOpenLowDelta[lookup] +=row[lookup]
          totalProfLongListWithHighOpenLowDelata[lookup].append(row[lookup])
        # if row['z-score-volume'] >= 2:
        #   totalProfLongWithHighVolume[lookup] +=row[lookup]
        #   totalProfLongListWithHighVolume[lookup].append(row[lookup])
      


###################
#Print Stats

def whoIsLargest(a,b,c,d):
  if a > b and a > c and a > d:
    return "no filter"
  if b > a and b > c and b > d:
    return "small open low delta"
  if c > a and c > b and c > d:
    return "large open low delta"
  if d > a and d > b and d > c:
    return "high volume"
  return "all are equal"
def whoIsSmallest(a,b,c,d):
  if a < b and a < c and a < d:
    return "no filter"
  if b < a and b < c and b < d:
    return "small open low delta"
  if c < a and c < b and c < d:
    return "large open low delta"
  if d < a and d < b and d < c:
    return "high volume"
  return "all are equal"
  
# for k, v in totalProfLong.items():
#   print(k, "any z-score", v, "filtered out with only low open low delta", totalProfLongWithLowOpenLowDelta[k], "diff", v - totalProfLongWithLowOpenLowDelta[k])

#Note total doesn't seem to matter much because the filters force less trades
for k, v in totalProfLongList.items():
  print("--------------------")
  print(k, "first is no filter, second is filtered out with only low open low delta")
  print("total", sum(v), "mean",  sum(v)/len(v), "std", statistics.stdev(v), "median", statistics.median(v), "max", max(v), "min", min(v))
  print("total", sum(totalProfLongListWithLowOpenLowDelata[k]) ,"mean",  sum(totalProfLongListWithLowOpenLowDelata[k])/len(totalProfLongListWithLowOpenLowDelata[k]), "std", statistics.stdev(totalProfLongListWithLowOpenLowDelata[k]), "median", statistics.median(totalProfLongListWithLowOpenLowDelata[k]), "max", max(totalProfLongListWithLowOpenLowDelata[k]), "min", min(totalProfLongListWithLowOpenLowDelata[k]))
  print("total", sum(totalProfLongListWithHighOpenLowDelata[k]) ,"mean",  sum(totalProfLongListWithHighOpenLowDelata[k])/len(totalProfLongListWithHighOpenLowDelata[k]), "std", statistics.stdev(totalProfLongListWithHighOpenLowDelata[k]), "median", statistics.median(totalProfLongListWithHighOpenLowDelata[k]), "max", max(totalProfLongListWithHighOpenLowDelata[k]), "min", min(totalProfLongListWithHighOpenLowDelata[k]))
  print("total", sum(totalProfLongListWithHighVolume[k]) ,"mean",  sum(totalProfLongListWithHighVolume[k])/len(totalProfLongListWithHighVolume[k]), "std", statistics.stdev(totalProfLongListWithHighVolume[k]), "median", statistics.median(totalProfLongListWithHighVolume[k]), "max", max(totalProfLongListWithHighVolume[k]), "min", min(totalProfLongListWithHighVolume[k]))
  mean_ = whoIsLargest(sum(v)/len(v), sum(totalProfLongListWithLowOpenLowDelata[k])/len(totalProfLongListWithLowOpenLowDelata[k]), sum(totalProfLongListWithHighOpenLowDelata[k])/len(totalProfLongListWithHighOpenLowDelata[k]), sum(totalProfLongListWithHighVolume[k])/len(totalProfLongListWithHighVolume[k]))
  std_  = whoIsSmallest(statistics.stdev(v), statistics.stdev(totalProfLongListWithLowOpenLowDelata[k]), statistics.stdev(totalProfLongListWithHighOpenLowDelata[k]), statistics.stdev(totalProfLongListWithHighVolume[k]))
  print("winners", "---mean---" , mean_, "---std---", std_)
# print(len(totalProfLongList))




In [32]:
totalProfLongList

defaultdict(list, {})

In [ ]:
index

21917

In [ ]:
df[df['z-score'] > 4.4]['z-score'].describe()

count    136.000000
mean       6.292723
std        2.182943
min        4.402684
25%        4.894591
50%        5.632471
75%        6.801954
max       18.694264
Name: z-score, dtype: float64

In [ ]:
df['z-score'].describe()

count    12348.000000
mean         0.038106
std          1.015914
min         -1.441537
25%         -0.622101
50%         -0.220376
75%          0.416632
max          9.933889
Name: z-score, dtype: float64

So I would like to visualize the spots that show this happening, just to see if I can spot a pattern that might be helpful to deal with. Rather than blindly coidng something.


In [ ]:
from datetime import datetime, timedelta, time
from   collections import defaultdict
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from   inspect import currentframe, getframeinfo
import requests
from   time import sleep
import os
import sys
import re

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def plotPandasDFWithCanldes(dataFrame_):

   fig = go.Figure(data=[go.Candlestick(x=dataFrame_['timeOnly'],

                open=dataFrame_['open'],

                high=dataFrame_['high'],

                low=dataFrame_['low'],

                close=dataFrame_['close'])])
   fig.update_xaxes(title_text='Date')

   fig.update_yaxes(title_text='Prices')

   fig.update_layout(xaxis_rangeslider_visible=True, # Set Range Slider Bar

                 title = 'Not bad') # Set Title

   fig.show()

In [ ]:
z_score_ = 4.4
print(":where is this going")

indexThatMatter = defaultdict(int)

for index, row in df.iterrows():
  if z_score_ < row['z-score']:
    indexThatMatter[index]=1

print(len(indexThatMatter))

count = 0
for index, row in df.iterrows():
  if indexThatMatter[index] == 1 and row['close'] - row['open'] > 0:
    print(index, "zscpre", row['z-score'])
    print("close", row['close'], "open", row['open'], "close-open", row['close'] - row['open'] , row['close'] - row['open'] > 0)

    min_df = df.iloc[[i for i in range(index, index+10)]]

    row1 = min_df.iloc[0]
    print("are rows, equal", row1.equals(row))


    plotPandasDFWithCanldes(min_df)
    count+=1
    if count == 30: break





:where is this going
136
1128 close 1.13517 open 1.12836 close-open 0.006809999999999983 True
are rows, equal False


1129 close 1.1387 open 1.13517 close-open 0.003530000000000033 True
are rows, equal False


1132 close 1.14284 open 1.13599 close-open 0.006850000000000023 True
are rows, equal False


1147 close 1.14708 open 1.14199 close-open 0.005090000000000039 True
are rows, equal False


1204 close 1.13194 open 1.12716 close-open 0.0047800000000000065 True
are rows, equal False


...

13044 close 1.13641 open 1.12916 close-open 0.007249999999999979 True
are rows, equal False


13449 close 1.123 open 1.11781 close-open 0.005190000000000028 True
are rows, equal False


13600 close 1.09424 open 1.08808 close-open 0.006160000000000165 True
are rows, equal False


13747 close 1.1045 open 1.10006 close-open 0.0044399999999999995 True
are rows, equal False


13953 close 1.10374 open 1.0985 close-open 0.005239999999999911 True
are rows, equal False


In [ ]:
z_score_ = 4.4
print(":where is this going")

start_index = df[df['z-score'] > 4.5].index.tolist()

count = 0
for index in start_index:
  if df.loc[index, 'close'] - df.loc[index, 'open'] > 0:
    #print(index, "zscpre", row['z-score'])
    #print("close", row['close'], "open", row['open'], "close-open", row['close'] - row['open'] , row['close'] - row['open'] > 0)

    min_df =  df.loc[index : index + 10] #df.iloc[[i for i in range(index, index+10)]]

    print(index)

    plotPandasDFWithCanldes(min_df)
    count+=1
    if count == 50: break





:where is this going
1128


1129


1132


1147


1204


...

17844


17910


17922


17929


18127


In [ ]:
start_index = df[df['z-score'] > 4.5].index.tolist()
start_index

[1030,
 1128,
 1129,
 1132,
 1147,
 1154,
 1160,
 1204,
 1217,
 1218,
 1219,
 1246,
 1248,
 1285,
 1323,
 1330,
 1338,
 1348,
 1351,
 1354,
 1356,
 1367,
 1369,
 1372,
 1378,
 1383,
 1401,
 1469,
 1940,
 2057,
 2123,
 2508,
 2653,
 2656,
 2765,
 2779,
 5256,
 5257,
 5259,
 5985,
 6360,
 7193,
 7290,
 7508,
 9068,
 11692,
 11918,
 13044,
 13274,
 13394,
 13437,
 13449,
 13535,
 13558,
 13600,
 13747,
 13953,
 14157,
 14245,
 14358,
 14406,
 14586,
 14603,
 15293,
 15439,
 15532,
 15631,
 15632,
 15782,
 15871,
 15898,
 15924,
 15943,
 16026,
 16041,
 16188,
 16260,
 16477,
 16502,
 16672,
 16677,
 16797,
 16806,
 16836,
 16974,
 16987,
 17040,
 17041,
 17165,
 17364,
 17518,
 17557,
 17678,
 17707,
 17757,
 17767,
 17844,
 17910,
 17921,
 17922,
 17929,
 18127,
 18146,
 18182,
 18185,
 18186,
 18228,
 18231,
 18396,
 18495,
 18726,
 18805,
 19241,
 19283,
 19335,
 19336,
 19813,
 19892,
 19897,
 19952,
 19953,
 20081,
 20120,
 20819,
 20933]